In [1]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.docstore.document import Document

import os

In [ ]:
! pip install sentence-transformers chromadb rank_bm25

In [2]:
documents = []
length_set = set()
count = 0
for filename in os.listdir('../../crawler/data/thutuchanhchinh_preprocessed2'):
    text_loader = TextLoader('../../crawler/data/thutuchanhchinh_preprocessed2/' + filename, encoding='utf-8')
    chunks = text_loader.load()[0].page_content.split('\n\n')
    length_set.add(len(chunks))
    if len(chunks) > 7:
        continue
    for chunk in chunks:
        padding_sentence = chunk.split('::')[0].strip()
        doc = Document(page_content=padding_sentence, metadata={'filename': filename, 'full_text':chunk})
        documents.append(doc)

In [3]:
len(documents)

12698

In [4]:
# Save the documents list with pickle
import pickle

with open('../documents.pkl', 'wb') as f:
    pickle.dump(documents, f)

In [5]:
HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")

/home/tranhuy/Desktop/Project/cantho-public-services/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tranhuy/Desktop/Project/cantho-public-services/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/tranhuy/Desktop/Project/cantho-public-services/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/tranhuy/Desktop/Project/cantho-public-se

In [6]:
vectorstore = Chroma.from_documents(documents, HF_EMBEDDING, persist_directory="../data2")

## BM25 embedding for administrative procedures

In [ ]:
## Initialize the retriever with all documents corpus
from langchain.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

In [ ]:
bm25_retriever.get_relevant_documents(question)